# Additional materials
This file contains the source code, comments to it and instructions for building.
It is also assumed, that the user already has [python](https://www.python.org/downloads/) downloaded and [pip](https://pip.pypa.io/en/stable/installation/) installed. If you are using other package managers such as Homebrew or Conda, you'll have to ensure that all used libraries are installed on your device

## Getting and processing data for the graph
As a result of human imperfection, there was a need to process data automatically. This removes the human factor and make the work way more easier. However, the creation of such a system can be very laborious, long-drawn-out, requiring patience and, in total, ungrateful. But in all we can get a very pretty result))

### Installing libraries
I used 3 libraries to handle, parse and process data, respectively, [Requests](https://pypi.org/project/requests/), [Beautiful Soup](https://pypi.org/project/beautifulsoup4/) and [Pandas](https://pypi.org/project/pandas/)

To construct and compute my graph I used [Networkx](https://pypi.org/project/networkx/) – a tool for working with complex networks, which is very helpful when we have many nodes and edges in our graph.

Finally, to export my graph into my $\rm\TeX$ cleanup, I used [tikzplotlib](https://pypi.org/project/tikzplotlib/). It will convert my graph object to a PGFPlot figure.

In [ ]:
%pip install beautifulsoup4
%pip install requests
%pip install pandas
#
%pip install networkx
#
%pip install tikzplotlib

### Importing libraries
Is there something to explain? :)

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#
import networkx as nx
#
import tikzplotlib

### Getting countries
It will be fabulous to have a list of land borders of European countries on Wiki in one, but they are separated, so the hard way you choose :(

We will first send a request to the [page](https://simple.wikipedia.org/wiki/List_of_European_countries) with the table of all European countries and check the status code (not necessary for this case, but useful in general):

In [5]:
r = requests.get(
    'https://simple.wikipedia.org/wiki/List_of_European_countries')
if r.status_code != 200:
    print('Error: website doesn\'t allow to download the source code')
    quit


We'll create a Beautiful Soup object by reading and parsing the text from our request and then find the part with the table in it.
Thereafter we have to delete (decompose) all data under tag 'sup' which stands for superscript which is used for footnotes in Wikipedia to "clear" countries name:

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find('table', {'class': "wikitable"})
for data in table(['sup']):
    data.decompose()

Finally we can build our dataframe. First we read the info from our table into a list of DataFrame objects and then build the DataFrame itself.
Then I deleted some columns I don't need and rename the column with countries to "Country". At last, I sorted the countries:

In [23]:
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df = df.drop(["Area(km²)", "Population(As of 2011)",
             "Population density(per km²)"], axis=1)
df.columns = ['Country', 'Capital']
df = df.sort_values(by=['Country'])


You can easily check your DataFrame just by printing the object or using the method "head" to print the n first rows:

In [24]:
print(df.head())

   Country           Capital
1  Albania            Tirana
2  Andorra  Andorra la Vella
0  Armenia           Yerevan
3  Austria            Vienna
4  Belarus             Minsk
